# 텍스트 데이터 전처리

### 패키지 임포트

In [1]:
import numpy as np
import matplotlib.pylab as plt

In [43]:
from __future__ import print_function, unicode_literals

import datetime
import re
import ujson
import zipfile

import pandas_datareader.data as web

from dateutil.parser import parse
from konlpy.utils import pprint

from sklearn.model_selection  import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

### 데이터 파일 로드

In [47]:
zf = zipfile.ZipFile('news_005930.zip', 'r')
jsondata = ujson.loads(zf.read('news_005930.json'))

In [48]:
pprint(jsondata[0])

{datetime: 2016-12-28 16:25:00,
 source: 머니투데이 ,
 text: [머니투데이 오정은 기자 2016년 증시 폐장을 하루 앞두고 결산이 한창이다. 올해 중소형주 급락으로 국내 증권가의 펀드매니저와 개인 투자자 대부분이 마이너스 수익률을 기록한 반면 외국인은 한국 주식 투자성과가 짭짤했던 것으로 나타났다.,
        횡보한 코스피, 급락한 코스닥과 달리 외국인이 추종하는 모건스탠리캐피탈인터내셔널(MSCI) 코리아 지수가 강세를 보여서다.,
        배당락일인 28일 코스피 지수는 전일대비 17.68포인트(0.87%) 내린 2024.49에 마감했다. 기관이 4147억원을 순매도하며 지수 하락을 주도했다. 반면 코스닥은 '1월 효과' 기대감에 1.44% 강세를 나타내며 627.27에 마감했다.,
        전일 한국거래소는 올해 코스피 현금배당액이 지난해와 같다고 가정하면 12월 결산법인의 현금배당을 감안한 코스피 배당락 지수는 2009.49로 추정한 바 있다.,
        최창규 NH투자증권 투자전략2팀장은 "배당을 받은 주식 매물이 쏟아지며 배당락 효과가 나타났다"며 "1월 옵션만기까지 배당 관련 매도가 이어질 것으로 예상된다"고 말했다.,
        폐장을 하루 앞둔 이날까지 올해 코스피 상승률은 3.2%를 기록했다. 반면 외국인 투자자들이 주로 추종하는 MSCI 코리아 지수의 상승률은 9.9%를 나타냈다. 외국인 입장에서는 한국 증시가 꽤 괜찮았던 셈이다.,
        코스피와 MSCI 코리아 지수의 수익률 격차가 큰 이유는 올해 증시에서 대형주는 강세를 보인 반면 중소형주는 급락했기 때문이다. 이날까지 코스피 대형주 지수의 수익률은 5.6%를 나타낸 반면 중형주 지수는 -7.3%, 소형주 지수는 0.8%를 기록했고 코스닥은 8.1% 하락했다.,
        이은택 SK증권 연구원은 "MSCI 코리아 지수는 한국 증시에서 시가총액 상위 우량 대형주 위주로 구성돼 대형주 강세장의 수혜를 봤다"며 "외국

### 내용이 없는 기사 삭제

In [5]:
stopwords = ['[포토]', '[사진]', '[스팟]']
for w in stopwords:
    jsondata = filter(lambda jd: not re.compile(w).search(jd['title']), jsondata)

### 필요없는 문장 삭제

뉴스에서 '삼성전자'라는 키워드가 없는 문장을 삭제하고 나머지 문장을 하나의 문자열로 바꾼다.

In [6]:
keyword = re.compile('삼성')
for jd in jsondata:
    jd['text'] = ' '.join(filter(keyword.search, jd['text']))

In [7]:
jsondata = filter(lambda jd: (len(jd['text']) > 0), jsondata)

In [8]:
jsondata = filter(lambda jd: keyword.search(jd['title']), jsondata)

In [9]:
print(len(jsondata))

8557


### 날짜 추가

In [10]:
for jd in jsondata:
    jd['date'] = parse(jd['datetime']).strftime('%Y-%m-%d')

### 저장

In [11]:
f = open('news_005930_clean.json', 'w')
ujson.dump(jsondata, f)
f.close()

### 라벨링

In [12]:
sec = web.DataReader("KRX:005930", data_source='google', start='2000-01-01', end='2017-01-01')

In [13]:
sec["Return"] = sec.Close.pct_change()
sec.dropna(inplace=True)
sec.tail()

,Open,High,Low,Close,Volume,Return
Date,,,,,,
2016-12-23,1801000.0,1804000.0,1780000.0,1782000.0,162173,-0.014925
2016-12-26,1780000.0,1800000.0,1778000.0,1798000.0,96051,0.008979
2016-12-27,1799000.0,1810000.0,1793000.0,1799000.0,91981,0.000556
2016-12-28,1792000.0,1799000.0,1780000.0,1788000.0,132355,-0.006115
2016-12-29,1771000.0,1802000.0,1770000.0,1802000.0,139768,0.007830


In [14]:
with open('news_005930_clean.json') as f:
    jsondata_clean = ujson.load(f)

### 매매가 없는 날 삭제

In [15]:
len(jsondata_clean)

8557

In [16]:
jsondata_label = filter(lambda jd: jd['date'] in sec.index, jsondata_clean)

In [17]:
len(jsondata_label)

7694

In [18]:
for jd in jsondata_label:
    jd['return'] = sec.ix[jd['date']].Return 

### 저장

In [19]:
f = open('news_005930_label.json', 'w')
ujson.dump(jsondata_label, f)
f.close()

### 형태소 분석

In [20]:
with open('news_005930_label.json') as f:
    jsondata_label = ujson.load(f)

In [21]:
from konlpy.tag import Kkma
tagger = Kkma()

stop_pos = ['ECD', 'ECE', 'ECS', 'EFN', 'EPT', 'ETD', 'ETN',
            'JC', 'JKG', 'JKM', 'JKO', 'JKS', 'JX', 
            'MAG', 'NNB', 'SP', 'SF', 'SS', 'SW', 'VCP', 'XR', 'XSN', 'XSV', 'VXV', 'VV']

def tokenize(jd):
    pos = tagger.pos(jd['text'])
    
    if pos[-1][1] in stop_pos:
        pos[-1] = None
    
    pos = filter(None, pos)
    
    jd['token'] = pos

In [22]:
%%time
for i, jd in enumerate(jsondata_label):
    if i % 500 == 0:
        print(datetime.datetime.now(), i)
    tokenize(jd)

2017-01-11 15:24:42.016000 0
2017-01-11 15:26:27.517000 500
2017-01-11 15:27:53.790000 1000
2017-01-11 15:29:32.876000 1500
2017-01-11 15:31:05.481000 2000
2017-01-11 15:32:40.945000 2500
2017-01-11 15:34:07.447000 3000
2017-01-11 15:35:36.151000 3500
2017-01-11 15:37:00.422000 4000
2017-01-11 15:38:32.444000 4500
2017-01-11 15:40:03.984000 5000
2017-01-11 15:41:36.560000 5500
2017-01-11 15:43:04.669000 6000
2017-01-11 15:44:37.484000 6500
2017-01-11 15:46:06.707000 7000
2017-01-11 15:47:40.007000 7500
Wall time: 23min 30s


In [23]:
pprint(jsondata_label[0]['title'])

삼성, 중국서 '갤럭시C5 프로' 전파인증 획득…중국시장 탈환 '시동'


In [24]:
pprint(jsondata_label[0]['token'][:10])

[(삼성전자, NNG),
 (가, JKS),
 (중국, NNG),
 (시장, NNG),
 (을, JKO),
 (정조준, NNG),
 (하, XSV),
 (어, ECS),
 (개발, NNG),
 (하, XSV)]


### 저장

In [25]:
f = open('news_005930_token.json', 'w')
ujson.dump(jsondata_label, f)
f.close()

### 벡터 인코딩

In [26]:
with open('news_005930_token.json') as f:
    jsondata_token = ujson.load(f)

In [27]:
corpus = [' '.join([''.join(t) for t in jd['token']]) for jd in jsondata_token]

In [28]:
tfidv = TfidfVectorizer().fit(corpus)
X = tfidv.transform(corpus)
y = np.array([jd['return'] > 0 for jd in jsondata_token])[:, np.newaxis]

In [29]:
X.shape, y.shape

((7694, 24226), (7694L, 1L))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

### Naive Bayesian 모형

In [40]:
model1 = MultinomialNB()
model1.fit(X_train, y_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
print(classification_report(y_train, model1.predict(X_train)))

             precision    recall  f1-score   support

      False       0.97      0.30      0.46      3087
       True       0.64      0.99      0.78      3837

avg / total       0.79      0.68      0.63      6924



In [33]:
print(classification_report(y_test, model1.predict(X_test)))

             precision    recall  f1-score   support

      False       0.93      0.20      0.33       347
       True       0.60      0.99      0.75       423

avg / total       0.75      0.63      0.56       770



### 커널 서포트 벡터 머신

In [34]:
model2 = SVC(random_state=0, gamma=1, C=1.0).fit(X_train, y_train)
model2.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
print(classification_report(y_train, model2.predict(X_train)))

             precision    recall  f1-score   support

      False       0.99      0.93      0.96      3087
       True       0.95      0.99      0.97      3837

avg / total       0.96      0.96      0.96      6924



In [36]:
print(classification_report(y_test, model2.predict(X_test)))

             precision    recall  f1-score   support

      False       0.82      0.65      0.73       347
       True       0.76      0.88      0.81       423

avg / total       0.78      0.78      0.78       770



### Decision Tree

In [37]:
model3 = DecisionTreeClassifier().fit(X_train, y_train)

In [38]:
print(classification_report(y_train, model3.predict(X_train)))

             precision    recall  f1-score   support

      False       0.99      1.00      1.00      3087
       True       1.00      1.00      1.00      3837

avg / total       1.00      1.00      1.00      6924



In [39]:
print(classification_report(y_test, model3.predict(X_test)))

             precision    recall  f1-score   support

      False       0.57      0.55      0.56       347
       True       0.64      0.67      0.65       423

avg / total       0.61      0.61      0.61       770



In [ ]:
model4 = Sequential()
model4.add(Dense(100, input_dim=24226, init='normal', activation='relu'))
model4.add(Dense(30, init='normal', activation='relu'))
model4.add(Dense(2, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])